## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [25]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [26]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [27]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [28]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210309.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [29]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,739530,73300,208600,1021430,951168,0.931212,263111,2021-03-08
1,Aragón,130185,12600,32700,175485,155770,0.887654,50418,2021-03-08
2,Asturias,128435,12700,25200,166335,136277,0.819292,44999,2021-03-08
3,Baleares,65880,6500,28800,101180,86720,0.857086,24685,2021-03-08
4,Canarias,148560,14300,53700,216560,180030,0.831317,55962,2021-03-08
5,Cantabria,62745,5900,14300,82945,68512,0.825993,21219,2021-03-08
6,Castilla y Leon,285195,27300,59000,371495,314360,0.846203,102339,2021-03-08
7,Castilla La Mancha,190575,18600,50400,259575,205143,0.790303,73053,2021-03-08
8,Cataluña,651140,63400,191900,906440,789183,0.870640,207196,2021-03-08
9,C. Valenciana,340660,31700,124700,497060,426373,0.857790,120409,2021-03-08


..hacemos una copia para hacerle las transformaciones necesarias:

In [30]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [31]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [32]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [33]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [34]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [35]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,739530,73300,208600,1021430,951168,0.931212,263111,8/3/21,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,130185,12600,32700,175485,155770,0.887654,50418,8/3/21,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,128435,12700,25200,166335,136277,0.819292,44999,8/3/21,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,65880,6500,28800,101180,86720,0.857086,24685,8/3/21,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,148560,14300,53700,216560,180030,0.831317,55962,8/3/21,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,62745,5900,14300,82945,68512,0.825993,21219,8/3/21,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,285195,27300,59000,371495,314360,0.846203,102339,8/3/21,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,190575,18600,50400,259575,205143,0.790303,73053,8/3/21,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,651140,63400,191900,906440,789183,0.870640,207196,8/3/21,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,340660,31700,124700,497060,426373,0.857790,120409,8/3/21,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [36]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [37]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [38]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [39]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,9/3/21,Andalucía,739530,73300,208600,1021430,951168,0.931212,263111,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,9/3/21,Aragón,130185,12600,32700,175485,155770,0.887654,50418,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,9/3/21,Asturias,128435,12700,25200,166335,136277,0.819292,44999,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,9/3/21,Baleares,65880,6500,28800,101180,86720,0.857086,24685,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,9/3/21,Canarias,148560,14300,53700,216560,180030,0.831317,55962,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,9/3/21,Cantabria,62745,5900,14300,82945,68512,0.825993,21219,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,9/3/21,Castilla y Leon,285195,27300,59000,371495,314360,0.846203,102339,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,9/3/21,Castilla La Mancha,190575,18600,50400,259575,205143,0.790303,73053,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,9/3/21,Cataluña,651140,63400,191900,906440,789183,0.870640,207196,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,9/3/21,C. Valenciana,340660,31700,124700,497060,426373,0.857790,120409,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [40]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [41]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [42]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,8/3/21,Navarra,62745.0,5900.0,16200.0,84845,73784,0.869633,24241.0,NaN,7/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
896,8/3/21,País Vasco,177660.0,16000.0,54800.0,248460,175424,0.706045,54348.0,NaN,7/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
897,8/3/21,Ceuta,5490.0,600.0,2200.0,8290,7634,0.920869,1818.0,NaN,5/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
898,8/3/21,Melilla,4320.0,600.0,2200.0,7120,5964,0.837640,1652.0,NaN,6/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [43]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [44]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total pauta completada', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [45]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [46]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,9/3/21,Navarra,62745.0,5900.0,16200.0,84845,74287,0.875561,24646.0,NaN,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
916,9/3/21,País Vasco,177660.0,16000.0,54800.0,248460,181629,0.731019,54456.0,NaN,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
917,9/3/21,Ceuta,5490.0,600.0,2200.0,8290,8177,0.986369,2360.0,NaN,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
918,9/3/21,Melilla,4320.0,600.0,2200.0,7120,6306,0.885674,1717.0,NaN,8/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [47]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [48]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)